### Introduction. 

This notebook is a sample of some basic data science work we might perform for commerical reasons. The dataset today comes from the European Social Survey of 2016/2017 (https://www.kaggle.com/pascalbliem/european-social-survey-ess-8-ed21-201617). 
The goal is to try and guess country of residence (of which there are 23)  from individual responses on the survey.


This dataset was chosen for the demo because it was was relatively small but with many features. The goal of country classification also goes nicely with how people understand the world, especially today where global immigration and nationalism and constantly discussed.

In [13]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB


from collections import defaultdict

### Importing the data 

Here we read in the csv files as pandas dataframes. Normally, we would do much more expoloratory analysis and visualizations to make sure we know what we are doing, but I've skipped over that part for the sake of time.   

In [3]:
# reading in the data
main_df = pd.read_csv("ESS8e02.1_F1.csv")
var_df = pd.read_csv("variables.csv")


 # convenience dict for going between the abreviations and the full country names. Just a good thing to have on hand.
country_name_dict = {"AT": "Austria",                   
                     "BE": "Belgium", 
                     "CH": "Switzerland", 
                     "CZ": "Czech Republic", 
                    "DE": "Germany", 
                    "EE": "Estonia", 
                    "ES": "Spain", 
                    "FI": "Finland", 
                    "FR": "France", 
                    "GB": "United Kingdom", 
                    "HU": "Hungary", 
                    "IE": "Ireland", 
                    "IL": "Israel", 
                    "IS": "Iceland", 
                    "IT": "Italy", 
                    "LT": "Lithuania", 
                    "NL": "Netherlands", 
                    "NO": "Norway", 
                    "PL": "Poland", 
                    "PT": "Portugal", 
                    "RU": "Russia", 
                    "SE": "Sweden", 
                    "SI": "Slovenia"}


C:\Users\Jack\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (164) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


the dataset comes in a pretty typical format: there's one csv file filled with the actual data (main_df), and then there is a separate csv file (var_df) explaining what everything means while also having some metadata. The second file is incrediblly useful for feature selection: often times, it is better to focus on the right parts of the dataset rather than just focusing on the algorithm. 

As mentioned earlier, the european survey file is actually pretty small (less than 45,000 respondents) but it has 534 features per overall. 534 features is a lot. To pare things down somewhat, I decided to drop all columns that are countyr specific. I then went throught the var_df file manually and picked out some features that seemed the most promising. 

In [4]:

# We have to drop all columns from main_df that are country specific. 
question_labels = list(var_df["Name"])
country_specific = list(var_df["Country_specific"])


# crude loop to get the column indices to be droped
drop_list = []
for i in range(len(question_labels)): 
    if country_specific[i] == "yes": 
        drop_list.append(question_labels[i])
        
                
# Dropping from the main table
main_df.drop(labels = drop_list, axis = 1, inplace = True)

# and dropping from var_df. 
var_df.set_index("Name", inplace= True)
var_df.drop(labels = drop_list, inplace= True)
var_df.reset_index(inplace= True)


### Feature selection.


After going through var_df I decided on the following features to feed into the model, which are listed below. To the right are the abreviated names along with any codes that indicate missing/incomplete responses.  

As one would expect, number 7 (language spoken) is by far the most useful for predicting nationality. This makes sense. If you speak Norwegian on a daily basis ... you probably live in Norway. The others might have some predictive power, but probably to as lesser extent

In [ ]:

# 1) age/gender                                                                      agea (999), gndr (9)

# 2) citizen/born in the country                                                    ctzcntr(9),  brncntr(9)

# 3) education, paid work, last 7 days.                                             pdwrk,  edctn                            

# 4) Respondent or household member victim of burglary/ assault last 5 years       crmvct (7, 8, 9)

# 5) whether they voted in the last national election.                             vote (3, 7, 8, 9)          

# 6) daily internet use, in minutes.                                               netustm (6666, 7777, 8888, 9999)

# 7**) language spoken, first or second                                             lnghom1, lnghom2  (777, 888, 999)


# 8) Would vote  to remain member of European Union or leave                         vteurmmb (33, 44, 55, 65, 77, 88, 99)


# 9) ever been divorced                                                             dvrcdeva (7, 8, 9)
  
# 10) Improve knowledge/skills: course/lecture/conference, last 12 months              atncrse(7, 8, 9)


# 11) Years of full-time education completed                                           eduyrs (77, 88, 99)

# 12) Year in which last held a paid job                                                    pdjobyr(6666, 7777, 8888, 9999)

# 13) News about politics and current affairs, watching, reading or listening, in minutes     nwspol (7777, 8888, 9999)


# 14) Would vote to become member of European Union or remain outside         vteubcmb(33, 44, 55, 65, 77, 88, 99)


Down below is the block where we replace all of the missing/incompete codes with np.nan for standardization.

In [5]:
# tedious replacement of invalid values with np.NAN...Not all of this is neccesary, so feel free to skip this step depending
# on the model you want to run. 

main_df["agea"].replace(999, np.nan)

main_df["gndr"].replace(9, np.nan)

main_df["ctzcntr"].replace(9, np.nan)

main_df["brncntr"].replace(9, np.nan)

main_df["crmvct"].replace([7, 8, 9], np.nan)

main_df["vote"].replace([3, 7, 8, 9], np.nan)

main_df["netustm"].replace([3, 7, 8, 9], np.nan)

main_df["vteurmmb"].replace([33, 44, 55, 65, 77, 88, 99], np.nan)

main_df["vteubcmb"].replace([33, 44, 55, 65, 77, 88, 99], np.nan)



# language ones 
main_df["lnghom1"].replace([777, 888, 999], np.nan)

main_df["lnghom2"].replace([777, 888, 999], np.nan)



print("")


### Modeling. 


Now we get to the fun part where we actually apply some machine learning algorithms. For this demo we are going to be using the k-nearest neighbor algorithm along with the random forest algorithm. Also Spoiler alert, k-nearest neighbors is not going to do very well, but it demonstrates the importance of choosing the right approach.


An important consideration is that the random forest and k-nearest neighbors algorithms only understand numbers, so we have to convert string labels (such as "GER" or "AT") into numbers. To do this we will use skleans label encoder. 

In [6]:
le_language = LabelEncoder()
le_country = LabelEncoder()

le_language.fit(main_df["lnghom1"])
main_df["lnghom1"] = le_language.transform(main_df["lnghom1"])

le_language.fit(main_df["lnghom2"])
main_df["lnghom2"] = le_language.transform(main_df["lnghom2"])

# And now for the country coulmn
le_country.fit(main_df["cntry"])
main_df["cntry"] = le_country.transform(main_df["cntry"])

Now our preprocessing is finally done we can train the models with the selected features. 

In [7]:
# Actual feautures we will use
keep_cols = ["agea", "gndr", "ctzcntr", "brncntr", "pdwrk", "edctn", "crmvct", "netustm", "vteurmmb",
             "dvrcdeva", "atncrse", "eduyrs", "pdjobyr", "nwspol", "lnghom1", "lnghom2", "vteubcmb"]

# the target vector
target_vector = main_df["cntry"]


# keep only the columns that we want. 
feature_df = main_df.copy()
feature_df = feature_df.replace(np.nan, -1)
feature_df = feature_df[keep_cols]



# splitting the data into train and test. Standard 80/20 split.  
X_train, X_test, y_train, y_test = train_test_split(feature_df, target_vector, test_size=0.2, random_state=19)


# nearest neighbors classifier. 
nn_clf = KNeighborsClassifier(n_neighbors= 3)

nn_clf.fit(X_train, y_train)

nn_y_hat_train = nn_clf.predict(X_train)
nn_y_hat_test = nn_clf.predict(X_test)

print("train accuracy for knn:", np.mean(y_train == nn_y_hat_train))
print("test accuracy for knn:", np.mean(y_test == nn_y_hat_test))
print("\n")




# Making our randoom forest and choosing the hyperparameters.
rand_forest_clf = RandomForestClassifier(criterion= "entropy", 
                                         n_estimators= 500, max_depth = 20, min_samples_split = 5, min_samples_leaf = 2)


rand_forest_clf.fit(X_train, y_train)

rand_forest_y_hat_train = rand_forest_clf.predict(X_train)
rand_forest_y_hat_test = rand_forest_clf.predict(X_test)

print("train accuracy for random forest:", np.mean(y_train == rand_forest_y_hat_train))
print("test accuracy for random forest:", np.mean(y_test == rand_forest_y_hat_test))


train accuracy for knn: 0.6666478920836971
test accuracy for knn: 0.39851317864383873


train accuracy for random forest: 0.9752738742290686
test accuracy for random forest: 0.9109033566118495


The random forest classifier is clearly superior for this task. This is because k-nearest neighbors only works for euclidean spaces, which is definitely not the case with the features we selected (most notably language, where the mapping is completely arbitrary). 

 One major advantage of random forests (and other tree-based algorithms) is that they are highly interpretable. 
 Below we can can clearly see which features are valued most highly by the random forest algorithm.


In [8]:
print("Feature Importance: \n")
for i in range(len(rand_forest_clf.feature_importances_)): 
    print(keep_cols[i], rand_forest_clf.feature_importances_[i] )

Feature Importance: 

agea 0.024777336101165795
gndr 0.005261927093398343
ctzcntr 0.006737325337363704
brncntr 0.010888378427186875
pdwrk 0.0038332466041030517
edctn 0.002403172906963358
crmvct 0.003993131609881037
netustm 0.01866900095319973
vteurmmb 0.1540387640344755
dvrcdeva 0.003741303641734748
atncrse 0.0081725942370872
eduyrs 0.03463595339124655
pdjobyr 0.011676749301470349
nwspol 0.022851144609327485
lnghom1 0.5212071084088408
lnghom2 0.06817510467571278
vteubcmb 0.09893775866684312


We can see that, by far, the most predictive features are first language, followed by pro-EU sentiment (vteubcmb and vteurmmb). This implies are lot of basic demographic and lifestyle questions are pretty useless and that political sentiment is the better predictor, which I would not have guessed starting out.   


Another good approach for this problem is a naive bayes classifier. This is a little tricky though, as the sklearn implmentation we will be using only accepts binary features (such as 1/0 or yes/no), not categorical ones. To get around this, we will use one-hot encoding to convert some categorical features to binary. 

In [9]:
keep_cols = ["vteurmmb", "vteubcmb", "lnghom1", "lnghom2", "gndr"]

target_vector = main_df["cntry"]


# keep only the columns that we want. 
feature_df = main_df.copy()
feature_df = feature_df.replace(np.nan, -1)
feature_df = feature_df[keep_cols]



ohe = OneHotEncoder(categories='auto')

ohe.fit(feature_df)
feature_df = ohe.transform(feature_df)



# splitting the data into train and test. 
X_train, X_test, y_train, y_test = train_test_split(feature_df, target_vector, test_size=0.2, random_state=19)



nb_classifier = BernoulliNB()

nb_classifier.fit(X_train, y_train)

nb_y_hat_train = nb_classifier.predict(X_train)
nb_y_hat_test = nb_classifier.predict(X_test)


print("train accuracy for BernoulliNB:", np.mean(y_train == nb_y_hat_train))
print("test accuracy for BernoulliNB:", np.mean(y_test == nb_y_hat_test))
print("\n")

train accuracy for BernoulliNB: 0.9108395054774846
test accuracy for BernoulliNB: 0.9061725613877




The results are slightly dispointing as the naive bayes actually does slightly worse than a decision tree. 
Let's make some code to get a better look at the classification error. 

In [21]:
# Make some code to look at classification error

predicted_labels = nb_classifier.predict(feature_df)

# a default dict that default key of 0 (int)
misclass_dict = defaultdict(int)

#  We have to convert them back from numbers to the string abbreviations. Bit of a hassle 
main_df["cntry"] = le_country.inverse_transform(main_df["cntry"])
predicted_labels = le_country.inverse_transform(predicted_labels)


for count, cntry_name in enumerate(main_df["cntry"]): 
    
    # If there is a classification error..
    if cntry_name != predicted_labels[count]:
        
        # add it to the default dict and increment.
        misclass_dict[(cntry_name, predicted_labels[count])] += 1
        
        
main_df["cntry"] = le_country.transform(main_df["cntry"])
               

for error in misclass_dict:
    print("{} mistaken for {} on {} occasions".format(country_name_dict[error[0]], country_name_dict[error[1]], 
                                                      misclass_dict[error]))     


Austria mistaken for Germany on 1301 occasions
Austria mistaken for Italy on 1 occasions
Austria mistaken for Belgium on 8 occasions
Austria mistaken for Hungary on 6 occasions
Austria mistaken for Slovenia on 1 occasions
Austria mistaken for Spain on 3 occasions
Austria mistaken for Ireland on 1 occasions
Austria mistaken for France on 1 occasions
Austria mistaken for Czech Republic on 1 occasions
Austria mistaken for Lithuania on 1 occasions
Belgium mistaken for Netherlands on 819 occasions
Belgium mistaken for France on 575 occasions
Belgium mistaken for Ireland on 24 occasions
Belgium mistaken for Poland on 3 occasions
Belgium mistaken for Italy on 2 occasions
Belgium mistaken for Portugal on 3 occasions
Belgium mistaken for Germany on 2 occasions
Belgium mistaken for Austria on 3 occasions
Belgium mistaken for Spain on 5 occasions
Belgium mistaken for Hungary on 1 occasions
Switzerland mistaken for Israel on 27 occasions
Switzerland mistaken for Russia on 2 occasions
Czech Republi

We can clearly see that the classifier has a lot of trouble separating Germany/Austria, Belgium/Netherlands/France, and Russia/Estonia. This is because these countries have a lot of language overlap, and in these cases the classifier usually assigns them to whichever country is seen more in the dataset (as it should). 

Next, we will try to improve our results by turning the naive bayes classifier output into a new feature for our dataset. This is known as "boosting", and it is a key technique in data science. The hope is that since the naive bayes classifier output correlates highly with the country, the random forest can take that information into account when classifying based on the other features. Think of it as a very strong "hint" of how that datapoint should be classified.

In [ ]:
main_df["nb_output"] = nb_classifier.predict(feature_df)

In [28]:
# Same keep cols as before, only that we have 
keep_cols = ["agea", "ctzcntr", "brncntr", "pdwrk", "edctn", "crmvct", "netustm", "vteurmmb",
             "dvrcdeva", "atncrse", "eduyrs", "pdjobyr", "nwspol", "lnghom1", "lnghom2", "vteubcmb",  
             "nb_output"]


# Redoing all the random forest stuff from before. This is a litteral copy pase 

target_vector = main_df["cntry"]


# keep only the columns that we want. 
feature_df = main_df.copy()
feature_df = feature_df.replace(np.nan, -1)
feature_df = feature_df[keep_cols]



X_train, X_test, y_train, y_test = train_test_split(feature_df, target_vector, test_size=0.2, random_state=19)



# Making our randoom forest and choosing the hyperparameters.
rand_forest_clf = RandomForestClassifier(criterion= "entropy", 
                                         n_estimators= 500, max_depth = 20, min_samples_split = 5, min_samples_leaf = 2)


rand_forest_clf.fit(X_train, y_train)

rand_forest_y_hat_train = rand_forest_clf.predict(X_train)
rand_forest_y_hat_test = rand_forest_clf.predict(X_test)

print("train accuracy for random forest:", np.mean(y_train == rand_forest_y_hat_train))
print("test accuracy for random forest:", np.mean(y_test == rand_forest_y_hat_test))

train accuracy for random forest: 0.9656706750401307
test accuracy for random forest: 0.9201396710970939


We did slightly better. Yay. 


If we were going to continue this project, we would investigate the main features that separate our main clusters of confusion (such as Germany/Austria). To accomplish this we would do more exploratory analysis, going feature by feature and making visualizations. We might also look into doing more advanced feature engineering/feature selection by doing a PCA.

Overall, this project could progress in many different ways and there is room for several different approaches.  